## Metrics calculation

Based on attentions, tf-idf, etc...

### 1) Configuration

In [ ]:
import pandas as pd
import numpy as np
import nltk
import string

STOPWORDS = nltk.corpus.stopwords.words('portuguese')

FOLDER = './results/'

COUNTS = {}
for i, row in pd.read_csv(f'{FOLDER}/result_counts.csv').iterrows():
    COUNTS[row['name']] = row['count']
    
token_metrics = []
COUNTS

### 2) Words metrics

In [ ]:
words_metrics = {}

#### Attention (General and by group)

In [ ]:
words_attention_df = pd.read_csv(f'{FOLDER}/words_attention.csv', sep=';')
words_attention_df['tokens'] = words_attention_df.tokens.apply(lambda x: str(x).split(' '))
words_attention_df = words_attention_df.loc[
    (words_attention_df.word.str.len() > 1) &\
    (~words_attention_df.word.isin(STOPWORDS)) &\
    (~words_attention_df.word.isin(list(string.punctuation)))
] # Remove words in stopwords
words_attention_df

In [ ]:
for i, row in words_attention_df.iterrows():
    if row.word not in words_metrics:
        words_metrics[row.word] = {
            'absolute': 0.0,
            'absolute_correct': 0.0,
            'absolute_incorrect': 0.0,
            
            'relative': 0.0,
            'relative_correct': 0.0,
            'relative_incorrect': 0.0,
            
            'antichina_absolute': 0.0,
            'antichina_absolute_correct': 0.0,
            'antichina_absolute_incorrect': 0.0,
            'antichina_relative': 0.0,
            'antichina_relative_correct': 0.0,
            'antichina_relative_incorrect': 0.0,
            
            'antivacina_absolute': 0.0,
            'antivacina_absolute_correct': 0.0,
            'antivacina_absolute_incorrect': 0.0,
            'antivacina_relative': 0.0,
            'antivacina_relative_correct': 0.0,
            'antivacina_relative_incorrect': 0.0,
            
            'provacina_absolute': 0.0,
            'provacina_absolute_correct': 0.0,
            'provacina_absolute_incorrect': 0.0,
            'provacina_relative': 0.0,
            'provacina_relative_correct': 0.0,
            'provacina_relative_incorrect': 0.0,
            
            'tfidf': 0.0,
            
            'antichina_tfidf': 0.0,
            'antivacina_tfidf': 0.0,
            'provacina_tfidf': 0.0,
            
            'antichina_tfidf_correct': 0.0,
            'antivacina_tfidf_correct': 0.0,
            'provacina_tfidf_correct': 0.0,
            
            'antichina_tfidf_incorrect': 0.0,
            'antivacina_tfidf_incorrect': 0.0,
            'provacina_tfidf_incorrect': 0.0,
        }
    
    words_metrics[row.word]['absolute'] = (
        (
            (
                row.antichina_correct_w + row.antivacina_correct_w + row.provacina_correct_w +\
                row.antichina_incorrect_w + row.antivacina_incorrect_w + row.provacina_incorrect_w
            ) / COUNTS['total'] # Maybe this should be the sum of ALL WEIGHTS.
        ) * 100 
    )
    
    words_metrics[row.word]['absolute_correct'] = (
        (
            (
                row.antichina_correct_w + row.antivacina_correct_w + row.provacina_correct_w
            ) / COUNTS['total'] # Maybe this should be the sum of ALL WEIGHTS.
        ) * 100 
    )
    
    words_metrics[row.word]['absolute_incorrect'] = (
        (
            (
                row.antichina_incorrect_w + row.antivacina_incorrect_w + row.provacina_incorrect_w
            ) / COUNTS['total'] # Maybe this should be the sum of ALL WEIGHTS.
        ) * 100 
    )
    
    words_metrics[row.word]['relative'] = (
        (
            (
                row.antichina_correct_w + row.antivacina_correct_w + row.provacina_correct_w +\
                row.antichina_incorrect_w + row.antivacina_incorrect_w + row.provacina_incorrect_w
            ) / (
                row.antichina_correct_n + row.antivacina_correct_n + row.provacina_correct_n +\
                row.antichina_incorrect_n + row.antivacina_incorrect_n + row.provacina_incorrect_n
            )
        ) * 100 
    )
    
    words_metrics[row.word]['relative_correct'] = (
        (
            (
                row.antichina_correct_w + row.antivacina_correct_w + row.provacina_correct_w
            ) / (
                row.antichina_correct_n + row.antivacina_correct_n + row.provacina_correct_n +\
                row.antichina_incorrect_n + row.antivacina_incorrect_n + row.provacina_incorrect_n
            )
        ) * 100 
    )  
    
    words_metrics[row.word]['relative_incorrect'] = (
        (
            (
                row.antichina_incorrect_w + row.antivacina_incorrect_w + row.provacina_incorrect_w
            ) / (
                row.antichina_correct_n + row.antivacina_correct_n + row.provacina_correct_n +\
                row.antichina_incorrect_n + row.antivacina_incorrect_n + row.provacina_incorrect_n
            )
        ) * 100 
    )
    
    
    # Now calculate for each group
    
    for group in ['antichina', 'antivacina', 'provacina']:
        
        words_metrics[row.word][f'{group}_absolute'] = (
            (
                (
                    row[f'{group}_correct_w'] + row[f'{group}_incorrect_w']
                ) / COUNTS[group] # Maybe this should be the sum of ALL WEIGHTS.
            ) * 100 
        )
        
        words_metrics[row.word][f'{group}_absolute_correct'] = (
            (
                (
                    row[f'{group}_correct_w']
                ) / COUNTS[group] # Maybe this should be the sum of ALL WEIGHTS.
            ) * 100 
        )
        
        words_metrics[row.word][f'{group}_absolute_incorrect'] = (
            (
                (
                    row[f'{group}_incorrect_w']
                ) / COUNTS[group] # Maybe this should be the sum of ALL WEIGHTS.
            ) * 100 
        )
        
        if (row[f'{group}_correct_n'] + row[f'{group}_incorrect_n']) == 0:
            break
        
        words_metrics[row.word][f'{group}_relative'] = (
            (
                (
                    row[f'{group}_correct_w'] + row[f'{group}_incorrect_w']
                ) / (
                    row[f'{group}_correct_n'] + row[f'{group}_incorrect_n']
                )
            ) * 100 
        )

        words_metrics[row.word][f'{group}_relative_correct'] = (
            (
                (
                    row[f'{group}_correct_w']
                ) / (
                    row[f'{group}_correct_n'] + row[f'{group}_incorrect_n']
                )
            ) * 100 
        )

        words_metrics[row.word][f'{group}_relative_incorrect'] = (
            (
                (
                    row[f'{group}_incorrect_w']
                ) / (
                    row[f'{group}_correct_n'] + row[f'{group}_incorrect_n']
                )
            ) * 100 
        )

In [ ]:
words_metrics_df = pd.DataFrame(words_metrics).T
words_metrics_df.insert(0, column='word', value=words_metrics_df.index)
words_metrics_df.to_csv(f'{FOLDER}/words_metrics.csv', index=None)
words_metrics_df.sample(5)

#### TF-IDF (General and by group)

In [ ]:
tfidf_weights = pd.read_csv(f'{FOLDER}/tfidf_weights.csv', sep=';', index_col=0)
tfidf_weights

In [ ]:
from ast import literal_eval

tfidf_indices = pd.read_csv(f'{FOLDER}/tfidf_indices.csv', sep=';', index_col=0)
tfidf_indices.antichina = tfidf_indices.antichina.apply(lambda x: literal_eval(x))
tfidf_indices.antivacina = tfidf_indices.antivacina.apply(lambda x: literal_eval(x))
tfidf_indices.provacina = tfidf_indices.provacina.apply(lambda x: literal_eval(x))

tfidf_indices

In [ ]:
for _, row in tfidf_weights.iterrows():
    
    try:
    
        words_metrics[row.name]['tfidf'] = np.mean([row[col] for col in tfidf_weights.columns])

        for group in ['antichina', 'antivacina', 'provacina']:

            group_tfidf_values = [row[index] for index in tfidf_indices[group][0]]

            words_metrics[row.name][f'{group}_tfidf'] = np.mean(
                group_tfidf_values
            )

            words_metrics[row.name][f'{group}_tfidf_correct'] = (
                np.sum([row[index] for index in tfidf_indices[group][1]]) / len(group_tfidf_values)
            )

            words_metrics[row.name][f'{group}_tfidf_incorrect'] = (
                np.sum([row[index] for index in tfidf_indices[group][2]]) / len(group_tfidf_values)
            )
    
    except:
        print('Error with word:', row.name)
        raise

#### Save DataFrame

In [ ]:
words_metrics_df = pd.DataFrame(words_metrics).T
words_metrics_df.insert(0, column='word', value=words_metrics_df.index)
words_metrics_df.to_csv(f'{FOLDER}/words_metrics.csv', index=None)
words_metrics_df.sample(5)